### Whisp a feature collection

Setup
- NB use a virtual environment to avoid altering your python environment (https://docs.python.org/3/tutorial/venv.html)

In [1]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [2]:
# Install openforis-whisp (if not already installed)
!pip install --pre openforis-whisp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import openforis_whisp as whisp

2025-04-04 14:56:20,705 - INFO - NumExpr defaulting to 2 threads.


Get a feature collection

In [3]:
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
print (GEOJSON_EXAMPLE_FILEPATH)

../tests/fixtures/geojson_example.geojson


Whisp it

In [4]:
df_formatted_stats = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Reading GeoJSON file from: /home/sepal-user/whisp/tests/fixtures/geojson_example.geojson
['Area', 'European_Primary_Forest', 'GLC_FCS30D_TC_2022', 'GLC_FCS30D_crop_2022', 'IFL_2020', 'IIASA_planted_plantation', 'Cocoa_bnetd', 'Oil_palm_Descals', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_TC', 'ESRI_2023_crop', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_lo

In [5]:
df_formatted_stats = whisp.whisp_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Reading GeoJSON file from: /home/sepal-user/whisp/tests/fixtures/geojson_example.geojson
['Area', 'European_Primary_Forest', 'GLC_FCS30D_TC_2022', 'GLC_FCS30D_crop_2022', 'IFL_2020', 'IIASA_planted_plantation', 'Cocoa_bnetd', 'Oil_palm_Descals', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_TC', 'ESRI_2023_crop', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_lo

Display table

In [6]:
df_formatted_stats

,geo,Admin_Level_1,Area,Centroid_lat,Centroid_lon,Cocoa_2023_FDaP,Cocoa_ETH,Cocoa_FDaP,Cocoa_bnetd,Country,...,TMF_deg_2022,TMF_deg_2023,TMF_deg_after_2020,TMF_deg_before_2020,TMF_plant,TMF_regrowth_2023,TMF_undist,Unit,plotId,ProducerCountry
0,"{'type': 'Polygon', 'coordinates': [[[-1.61283...",Ashanti Region,1.939,6.159540,-1.611942,0.000,0.000,0.000,0.000,GHA,...,0,0.000,0.000,0.637,0.000,0.803,0.000,ha,1,GH
1,"{'type': 'Polygon', 'coordinates': [[[-1.64615...",Ashanti Region,4.152,6.104735,-1.644732,0.003,0.010,0.010,0.000,GHA,...,0,0.000,0.000,1.971,0.000,0.017,0.000,ha,2,GH
2,"{'type': 'Polygon', 'coordinates': [[[-2.15951...",Western Region,16.600,5.981149,-2.157144,0.000,0.000,0.000,0.000,GHA,...,0,0.089,0.089,0.738,0.000,0.000,15.862,ha,3,GH
3,"{'type': 'Polygon', 'coordinates': [[[103.9514...",South Sumatra,31.213,-3.054668,103.956096,0.000,0.000,0.000,0.000,IDN,...,0,0.000,0.000,0.000,31.213,0.000,0.000,ha,4,ID
4,"{'type': 'Polygon', 'coordinates': [[[103.9694...",South Sumatra,1.964,-3.068831,103.970371,0.000,0.000,0.000,0.000,IDN,...,0,0.000,0.000,0.520,0.000,0.316,0.000,ha,5,ID
5,"{'type': 'Polygon', 'coordinates': [[[103.9731...",South Sumatra,12.725,-3.082922,103.975182,0.000,0.000,0.000,0.000,IDN,...,0,0.000,0.000,2.490,0.000,1.431,7.558,ha,6,ID
6,"{'type': 'Polygon', 'coordinates': [[[103.9749...",South Sumatra,20.882,-3.083808,103.977512,0.000,0.000,0.000,0.000,IDN,...,0,0.714,0.714,4.253,0.000,4.897,5.569,ha,7,ID
7,"{'type': 'Polygon', 'coordinates': [[[-4.10288...",Lagunes,8.279,5.711935,-4.101646,0.072,0.000,0.376,3.013,CIV,...,0,0.000,0.000,1.795,0.000,3.282,0.006,ha,8,CI
8,"{'type': 'Polygon', 'coordinates': [[[-4.08767...",Lagunes,1.981,5.673811,-4.086848,0.276,1.494,0.011,0.006,CIV,...,0,0.000,0.000,0.803,0.000,0.715,0.000,ha,9,CI
9,"{'type': 'Polygon', 'coordinates': [[[-4.12062...",District Autonome D'Abidjan,3.797,5.572136,-4.119589,0.000,0.000,0.004,0.050,CIV,...,0,0.000,0.000,0.132,0.000,2.857,0.000,ha,10,CI


In [7]:
# Define the output folder
out_directory = Path.home() / 'module_results/whisp'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_stats.csv'

# Save the CSV file
df_formatted_stats.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: /home/sepal-user/module_results/whisp/whisp_output_table_stats.csv


Calculate risk category

In [7]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

TypeError: unsupported operand type(s) for /: 'float' and 'str'

Display table with risk columns

In [ ]:
df_w_risk

Export table to CSV

In [ ]:
# Define the output folder
out_directory = Path.home() / 'Downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_w_risk.csv'

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_table.geojson'

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.
print(f"GeoJSON file saved to: {geojson_output_file}")